In [2]:
import pandas as pd
import os
from scipy import spatial
from glob import glob

# [合併車禍csv原始資料]

In [18]:
def A1A2()->pd.DataFrame():
    print('開始合併車禍資料')
    path = "A1A2原始資料/"
    files = os.listdir(path)
    data = None
    for file in files:        
        if ('csv' in file) and ('result' not in file) and  ('station' not in file):
            if str(type(data))!="<class 'pandas.core.frame.DataFrame'>":
                data = pd.read_csv(f'{path}{file}', encoding='utf-8-sig', low_memory=False)
                continue 
            new = pd.read_csv(f'{path}{file}', encoding='utf-8-sig', low_memory=False)
            data = pd.concat([data, new], axis=0)
        else:
            continue
    
    ### 把沒有經緯度的raw去除
    data = data[data['經度'].isnull()==False]
    data = data[data['經度']!='經度']
    print('車禍資料合併完成')
    return data

# [類別資料重新編碼]

In [19]:
def recoding(df:pd.DataFrame())->pd.DataFrame():
    print('開始編碼類別資料')
    
    df['發生年度'] = df['發生年度'].astype('int')
    df['發生日期'] = df['發生日期'].astype('int')
    df['發生時間'] = df['發生時間'].astype('int')
    df['發生月份'] = df['發生月份'].astype('int')
    df['發生日期'] = df['發生日期'].astype('int')
    df['經度'] = df['經度'].astype('str')
    df['緯度'] = df['緯度'].astype('str')
    
    print('天氣recoding')
    #天氣recoding
    dict = {"雨" : '雨', 
            "風沙" : '特殊', 
            "強風": '特殊', 
            "陰" : '陰', 
            "雪": '特殊', 
            "暴雨": '特殊', 
            "霧或煙": '特殊', 
            "晴": '晴'}
    
    df2=df.replace({"天候名稱": dict})

    #天氣get dummies
    df_weather = pd.get_dummies(df2.天候名稱, prefix='weather', dtype='int64')
    df_weather.reset_index(drop=True)
    
    print('光線recoding')
    #光線recoding
    dict = {"日間自然光線" : '日間自然光線', 
            "夜間(或隧道、地下道、涵洞)有照明" : '夜間有照明', 
            "夜間(或隧道、地下道、涵洞)無照明": '夜間無照明', 
            "晨或暮光": '晨或暮光'}
    df3=df.replace({"光線名稱": dict})
    df_light = pd.get_dummies(df3.光線名稱, prefix='light', dtype='int64')
    df_light.reset_index(drop=True)
    
    print('道路類型recoding')
    #道路類型recoding
    dict = {"平交道" : '交岔路', 
            "交岔路" : '交岔路', 
            "單路部分": '單路部分', 
            "圓環廣場": '交岔路'}
    df4=df.replace({"道路型態大類別名稱": dict})

    df_roadtype = pd.get_dummies(df4.道路型態大類別名稱, prefix='roadtype', dtype='int64')
    df_roadtype.reset_index(drop=True)

    weather_road = [df_weather, df_roadtype]
    df_1 = pd.concat(weather_road, axis =0 ,ignore_index=False)
    
    print('道路乾溼recoding')
    #道路乾溼recoding
    dict = {"冰雪" : '濕潤', 
            "油滑" : '濕潤', 
            "泥濘": '濕潤', 
            "乾燥": '乾燥', 
            "濕潤": '濕潤'}
    df5=df.replace({"路面狀況-路面狀態名稱": dict})

    df_drywet = pd.get_dummies(df5["路面狀況-路面狀態名稱"], prefix='drywet', dtype='int64')
    
    print('道路坑洞recoding')
    #道路坑洞recoding
    dict = {"有坑洞" : '有坑洞', 
            "突出(高低)不平" : '有坑洞', 
            "無缺陷": '無缺陷', 
            "路面鬆軟": '有坑洞'}
    df6=df.replace({"路面狀況-路面缺陷名稱": dict})
    df_pit = pd.get_dummies(df6["路面狀況-路面缺陷名稱"], prefix='pit', dtype='int64')
    
    print('障礙物recoding')
    #障礙物recoding
    dict = {"無障礙物" : '無障礙物', 
            "有堆積物" : '有障礙物', 
            "其他障礙物": '有障礙物', 
            "道路工事(程)中": '有障礙物',        
            "路上有停車": '有障礙物'}
    df7=df.replace({"道路障礙-障礙物名稱": dict})
    df_obst = pd.get_dummies(df7["道路障礙-障礙物名稱"], prefix='obst', dtype='int64')
    
    print('號誌recoding')
    #號誌recoding
    dict = {"行車管制號誌" : '行車管制號誌', 
            "行車管制號誌(附設行人專用號誌)" : '行車管制號誌', 
            "閃光號誌": '閃光號誌',        
            "無號誌": '無號誌'}
    df8=df.replace({"號誌-號誌種類名稱": dict})
    df_sign = pd.get_dummies(df8["號誌-號誌種類名稱"], prefix='sign', dtype='int64')
    
    print('分隔線類型recoding')
    #分隔線recoding
    dict = {"單向禁止超車線" : '禁止超車線', 
            "雙向禁止超車線" : '禁止超車線', 
            "中央分向島": '中央分向島', 
            "無": '無', 
            "行車分向線": '行車分向線'}
    df9=df.replace({"車道劃分設施-分向設施大類別名稱": dict})
    df_line = pd.get_dummies(df9["車道劃分設施-分向設施大類別名稱"], prefix='line', dtype='int64')
    
    print('速限recoding')
    #速限recoding
    def my_recode(speed):
        if speed >= 110:
            return 110
        elif 110 > speed >= 100:
            return 100
        elif 100 > speed >= 80:
            return 80   
        elif 80 > speed >= 60:
            return 60    
        elif 60 > speed >= 50:
            return 50
        elif 50 > speed >= 40:
            return 40
        else:
            return 30
    df['速限_recode'] = df['速限-第1當事者'].apply(my_recode)
    df_speed = df['速限_recode']
    
    print('合併recoding表格')
    df_combined = df[['發生年度','發生月份','發生日期','發生時間','事故類別名稱','處理單位名稱警局層','發生地點','經度','緯度']]
    frames = [df_combined, df_weather, df_light, df_roadtype ,df_drywet ,df_pit ,df_obst ,df_sign ,df_line ,df_speed]
    df_all = pd.concat(frames, axis =1 ,ignore_index=False)
    
    print('recoding完成')
    return df_all

# [合併車流量]

In [20]:
def streamCombine(data:pd.DataFrame)->pd.DataFrame:
    print('開始合併車流資料')
    stream = pd.read_csv(f'合併車流量/106-110交通流量彙整.csv', low_memory=False)
    stream['總計_流量(PCU)'] = stream['總計_流量(PCU)'].str.findall('\d').str.join('').astype(int)
    stream['ID']=stream['年份'].astype(str)+stream['調查站_座標_經度'].astype(str)+stream['調查站_座標_緯度'].astype(str)
    stream = stream.groupby('ID').mean()
    
    print('建立車流站點KDTree')
    KDTree06 = spatial.KDTree(stream[stream['年份']==106][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree07 = spatial.KDTree(stream[stream['年份']==107][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree08 = spatial.KDTree(stream[stream['年份']==108][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree09 = spatial.KDTree(stream[stream['年份']==109][['調查站_座標_經度','調查站_座標_緯度']])
    KDTree10 = spatial.KDTree(stream[stream['年份']==110][['調查站_座標_經度','調查站_座標_緯度']])
    global c
    c=0
    def nearest(x: pd.DataFrame):
        if x.發生年度 == 2022:                      #事故民國111年，流量民國110年
            distance = KDTree10.query((x.經度, x.緯度),k=3)[0]
            index = KDTree10.query((x.經度, x.緯度),k=3)[1]
        elif x.發生年度 == 2021:                    #事故民國110年，流量民國109年
            distance = KDTree09.query((x.經度, x.緯度),k=3)[0]
            index = KDTree09.query((x.經度, x.緯度),k=3)[1]
        elif x.發生年度 == 2020:                    #事故民國109年，流量民國108年
            distance = KDTree08.query((x.經度, x.緯度),k=3)[0]
            index = KDTree08.query((x.經度, x.緯度),k=3)[1]
        elif x.發生年度 == 2019:                    #事故民國108年，流量民國107年
            distance = KDTree07.query((x.經度, x.緯度),k=3)[0]
            index = KDTree07.query((x.經度, x.緯度),k=3)[1]
        else:                                      #事故民國107年，流量民國106年
            distance = KDTree06.query((x.經度, x.緯度),k=3)[0]
            index = KDTree06.query((x.經度, x.緯度),k=3)[1]
        df = stream.iloc[index,:]
        df['與事故點距離'] = pd.Series(distance,index=df.index)
        df = df[['調查站_座標_經度','調查站_座標_緯度','與事故點距離','總計_流量(PCU)']]
        df = pd.concat([df.iloc[0,],df.iloc[1,],df.iloc[2,]])

        global c
        c+=1
        if c%100000==0:
            print(f"車禍點最近的車流站點:已計算{c}筆",'|',' ', end='')
        return df
    data = pd.concat([data, data.apply(nearest, axis=1)], axis = 1)
    data.columns = list(data.columns[:-12])+['調查站1_座標_經度','調查站1_座標_緯度','調查站1與事故點距離','調查站1總計_流量(PCU)','調查站2_座標_經度','調查站2_座標_緯度','調查站2與事故點距離','調查站2總計_流量(PCU)','調查站3_座標_經度','調查站3_座標_緯度','調查站3與事故點距離','調查站3總計_流量(PCU)']
    
    print('車流資料合併完成')
    return data

# [合併天氣]

In [21]:
def weatherCombine(data:pd.DataFrame)->pd.DataFrame:
    print('開始合併天氣資料')
    
    ### 合併各年份氣象站天氣資料
    years = [2018, 2019, 2020, 2021,2022]
    weather = None
    for year in years:        
        if str(type(data))!="<class 'pandas.core.frame.DataFrame'>":
            weather = pd.read_csv(f'./合併天氣/weather{year}.csv',encoding='utf-8-sig', low_memory=False)
            print(f'已讀取{year}年天氣資料')
            continue 
        new = pd.read_csv(f'./合併天氣/weather{year}.csv',encoding='utf-8-sig', low_memory=False)
        weather = pd.concat([weather, new], axis=0)
        print(f'已讀取{year}年天氣資料')
        
    print('開始讀取天氣站名與站點位置')
    ### 取出天氣站的站名與站點
    station_has_weather = pd.DataFrame({'location':weather['location'].unique()}).merge( weather.groupby('location')[['longitude','latitude']].mean(), how='left',on='location')
    origin_station_data = pd.read_csv('./合併天氣/stationData.csv', low_memory=False)[['name','location']]
    station_has_weather = station_has_weather.merge(origin_station_data, how='left', left_on='location', right_on='name')[['location_x', 'longitude', 'latitude','location_y']]
    station_has_weather.columns = ['name','long','lat','location']
    
    print('已取出有天氣資料的站點，以及站點的位置資料')
    
    #station
    station = station_has_weather
    station_has_weather = None

    
    print('開始將有天氣資料的站點建立KDtree')
    ### 計算事故點最近的三個氣象站
    KDTree = spatial.KDTree(station[['long','lat']])
    print('開始計算每個車禍點最近的3個氣象站點')
    global c
    c=0
    def nearest_weather_station(x: pd.DataFrame):
        TreeResult = KDTree.query((x.經度, x.緯度),k=3)
        distance = TreeResult[0]  #[測站1距離(經度*緯度),測站2距離(經度*緯度),測站3距離(經度*緯度)]
        index = TreeResult[1]     #[測站1在station的index,測站2在station的index, 測站3在station的index]

        first_station = station.iloc[index[0], [0,3]]     #[0,3]的column為測站名稱、測站地址
        second_station = station.iloc[index[1], [0,3]]
        third_station = station.iloc[index[2], [0,3]]

        first_distance = distance[0]
        second_distance = distance[1]
        third_distance = distance[2]

        ser = pd.Series([first_station[0],first_distance,second_station[0],second_distance,third_station[0],third_distance])

        global c
        c+=1
        if c%100000 == 0:
            print(f"車禍最近的3個氣象站點:已計算{c}筆| ",end='' )

        return ser

    data = pd.concat([data, data.apply(nearest_weather_station, axis=1)], axis = 1)
    data = data.rename(columns={0:'氣象站1',
                                1:'氣象站1距離',
                                2:'氣象站2',
                                3:'氣象站2距離',
                                4:'氣象站3',
                                5:'氣象站3距離',
                                })
    print('已合併與車禍點最近的三個天氣站點名稱')
    
    # 合併三個最近天氣站的天氣資料
    print('開始合併各天氣站的天氣資料')
    weather['date'] = weather['date'].str.findall('\d+').str.join('')
    weather['觀測時間(hour)'] = weather['觀測時間(hour)'].astype(str).apply(lambda x: '0' + x if len(x) <2 else '00' if x == '24' else x)
    weather['weather_concat'] = weather['date'] + weather['觀測時間(hour)'] + weather['location']
    
    print('開始合併最近的天氣站天氣資料')
    data['weather_concat_station1'] = data['發生日期'].astype(int).astype(str)+(data['發生時間']//10000).astype(int).astype(str).apply(lambda x: '0' + x if len(x) <2 else x)+data['氣象站1']
    data = data.merge(weather.rename(columns=dict(zip(weather.columns, weather.columns+'_station1'))), how='left', left_on='weather_concat_station1', right_on='weather_concat_station1')
    
    print('開始合併第二近的天氣站天氣資料')
    data['weather_concat_station2'] = data['發生日期'].astype(int).astype(str)+(data['發生時間']//10000).astype(int).astype(str).apply(lambda x: '0' + x if len(x) <2 else x)+data['氣象站2']
    data = data.merge(weather.rename(columns=dict(zip(weather.columns, weather.columns+'_station2'))), how='left', left_on='weather_concat_station2', right_on='weather_concat_station2')
    
    print('開始合併第三近的天氣站天氣資料')
    data['weather_concat_station3'] = data['發生日期'].astype(int).astype(str)+(data['發生時間']//10000).astype(int).astype(str).apply(lambda x: '0' + x if len(x) <2 else x)+data['氣象站3']
    data = data.merge(weather.rename(columns=dict(zip(weather.columns, weather.columns+'_station3'))), how='left', left_on='weather_concat_station3', right_on='weather_concat_station3')
    
    print('天氣資料合併完成')
    
    N = data[(data['氣溫(℃)_station1'].isnull()==True) & (data['氣溫(℃)_station2'].isnull()==True) & (data['氣溫(℃)_station3'].isnull()==True)].shape[0]
    print(f'共有 {N}筆車禍資料的三個天氣站均無氣溫資料')
    
    return data

# [合併人口密度資料]

In [22]:
def density(df: pd.DataFrame)->pd.DataFrame:
    print('開始合併人口密度資料')
    df['市/縣區'] = df["發生地點"].str.findall('..[市].*?[區]|..[縣].*?[鄉鎮市]')
    def _(l):
        try:
            l[0]
            return l[0]
        except Exception:
            return None

    df['市/縣區'] = df['市/縣區'].apply(_)

    df['年/地區'] = df['發生年度'].astype(str)+df['市/縣區']

    files = glob('./合併人口密度/opendata*N010.csv')

    df1 = pd.concat(
        (pd.read_csv(file, usecols=['statistic_yyy','site_id','population_density'], dtype={ 'statistic_yyy': str, 'site_id':str, 'population_density':str}) for file in files), ignore_index=True)
    df1.dropna(inplace=True)
    df1 = df1[df1['population_density']!='…']
    df1 = df1[df1['statistic_yyy']!='統計年']
    df1['西元年'] = df1['statistic_yyy'].map(int)+1912
    df1['年/地區'] = df1['西元年'].map(str)+df1['site_id']
    df = pd.merge(df, 
                 df1, 
                 on ='年/地區', 
                 how ='left')
    print('人口密度資料合併完成')
    return df

In [23]:
data = A1A2()                 #各年車禍原始資料合併
data = recoding(data)         #類別變數重編碼
data = streamCombine(data)    #合併車流資料
data = weatherCombine(data)   #合併天氣資料
data = density(data)          #合併人口密度資料
data.to_csv('positive_data.csv', encoding='utf-8-sig', mode='w',index=False)

開始合併車禍資料
車禍資料合併完成
開始編碼類別資料
天氣recoding
光線recoding
道路類型recoding
道路乾溼recoding
道路坑洞recoding
障礙物recoding
號誌recoding
分隔線類型recoding
速限recoding
合併recoding表格
recoding完成
開始合併車流資料
建立車流站點KDTree


C:\Users\student\AppData\Local\Temp\ipykernel_6464\1225651620.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  stream = stream.groupby('ID').mean()


車禍點最近的車流站點:已計算100000筆 |  車禍點最近的車流站點:已計算200000筆 |  車禍點最近的車流站點:已計算300000筆 |  車禍點最近的車流站點:已計算400000筆 |  車禍點最近的車流站點:已計算500000筆 |  車禍點最近的車流站點:已計算600000筆 |  車禍點最近的車流站點:已計算700000筆 |  車禍點最近的車流站點:已計算800000筆 |  車禍點最近的車流站點:已計算900000筆 |  車禍點最近的車流站點:已計算1000000筆 |  車禍點最近的車流站點:已計算1100000筆 |  車禍點最近的車流站點:已計算1200000筆 |  車禍點最近的車流站點:已計算1300000筆 |  車禍點最近的車流站點:已計算1400000筆 |  車禍點最近的車流站點:已計算1500000筆 |  車禍點最近的車流站點:已計算1600000筆 |  車禍點最近的車流站點:已計算1700000筆 |  車禍點最近的車流站點:已計算1800000筆 |  車禍點最近的車流站點:已計算1900000筆 |  車禍點最近的車流站點:已計算2000000筆 |  車禍點最近的車流站點:已計算2100000筆 |  車禍點最近的車流站點:已計算2200000筆 |  車禍點最近的車流站點:已計算2300000筆 |  車禍點最近的車流站點:已計算2400000筆 |  車禍點最近的車流站點:已計算2500000筆 |  車禍點最近的車流站點:已計算2600000筆 |  車禍點最近的車流站點:已計算2700000筆 |  車禍點最近的車流站點:已計算2800000筆 |  車禍點最近的車流站點:已計算2900000筆 |  車禍點最近的車流站點:已計算3000000筆 |  車禍點最近的車流站點:已計算3100000筆 |  車禍點最近的車流站點:已計算3200000筆 |  車禍點最近的車流站點:已計算3300000筆 |  車禍點最近的車流站點:已計算3400000筆 |  車禍點最近的車流站點:已計算3500000筆 |  車禍點最近的車流站點:已計算3600000筆 |  車禍點最近的車流站點:已計算3700000筆 |  車流資料合併完成
開始合併天氣資料
已讀取2018年天氣資料
已讀取2019年天氣資料
已讀取

In [24]:
data.shape

(3802331, 126)

In [25]:
data.head(10)

,發生年度,發生月份,發生日期,發生時間,事故類別名稱,處理單位名稱警局層,發生地點,經度,緯度,weather_晴,...,能見度(km)_station3,紫外線指數_station3,總雲量(0~10)_station3,海平面氣壓(gpm)_station3,市/縣區,年/地區,statistic_yyy,site_id,population_density,西元年
0,2018,1,20180101,300,A2,臺南市政府警察局,臺南市安平區安北路東側 / 臺南市安平區大平路,120.152935,22.999773,1,...,...,0,...,NaN,臺南市安平區,2018臺南市安平區,106,臺南市安平區,6009,2018
1,2018,1,20180101,300,A2,臺南市政府警察局,臺南市安平區安北路東側 / 臺南市安平區大平路,120.152935,22.999773,1,...,...,0,...,NaN,臺南市安平區,2018臺南市安平區,106,臺南市安平區,6009,2018
2,2018,1,20180101,300,A2,臺南市政府警察局,臺南市安平區安北路東側 / 臺南市安平區大平路,120.152935,22.999773,1,...,...,0,...,NaN,臺南市安平區,2018臺南市安平區,106,臺南市安平區,6009,2018
3,2018,1,20180101,300,A2,臺中市政府警察局,臺中市西屯區上石里西屯路二段273之1號前0.0公尺,120.644124,24.174996,1,...,...,...,...,...,臺中市西屯區,2018臺中市西屯區,106,臺中市西屯區,5698,2018
4,2018,1,20180101,300,A2,臺中市政府警察局,臺中市西屯區上石里西屯路二段273之1號前0.0公尺,120.644124,24.174996,1,...,...,...,...,...,臺中市西屯區,2018臺中市西屯區,106,臺中市西屯區,5698,2018
5,2018,1,20180101,400,A2,國道公路警察局,雲林縣林內鄉國道3號 253公里500.0公尺處南向中線,120.605676,23.762712,1,...,...,...,...,NaN,雲林縣林內鄉,2018雲林縣林內鄉,106,雲林縣林內鄉,485,2018
6,2018,1,20180101,400,A2,國道公路警察局,雲林縣林內鄉國道3號 253公里500.0公尺處南向中線,120.605676,23.762712,1,...,...,...,...,NaN,雲林縣林內鄉,2018雲林縣林內鄉,106,雲林縣林內鄉,485,2018
7,2018,1,20180101,400,A2,國道公路警察局,雲林縣林內鄉國道3號 253公里500.0公尺處南向中線,120.605676,23.762712,1,...,...,...,...,NaN,雲林縣林內鄉,2018雲林縣林內鄉,106,雲林縣林內鄉,485,2018
8,2018,1,20180101,400,A2,國道公路警察局,雲林縣林內鄉國道3號 253公里500.0公尺處南向中線,120.605676,23.762712,1,...,...,...,...,NaN,雲林縣林內鄉,2018雲林縣林內鄉,106,雲林縣林內鄉,485,2018
9,2018,1,20180101,400,A2,國道公路警察局,雲林縣林內鄉國道3號 253公里500.0公尺處南向中線,120.605676,23.762712,1,...,...,...,...,NaN,雲林縣林內鄉,2018雲林縣林內鄉,106,雲林縣林內鄉,485,2018


In [26]:
data.tail(10)

,發生年度,發生月份,發生日期,發生時間,事故類別名稱,處理單位名稱警局層,發生地點,經度,緯度,weather_晴,...,能見度(km)_station3,紫外線指數_station3,總雲量(0~10)_station3,海平面氣壓(gpm)_station3,市/縣區,年/地區,statistic_yyy,site_id,population_density,西元年
3802321,2022,11,20221112,194517,A1,宜蘭縣政府警察局,宜蘭縣冬山鄉鹿梅路前0.0公尺,121.744672,24.655476,1,...,...,...,...,NaN,宜蘭縣冬山鄉,2022宜蘭縣冬山鄉,110,宜蘭縣冬山鄉,663,2022
3802322,2022,11,20221112,194517,A1,宜蘭縣政府警察局,宜蘭縣冬山鄉鹿梅路前0.0公尺,121.744672,24.655476,1,...,...,...,...,NaN,宜蘭縣冬山鄉,2022宜蘭縣冬山鄉,110,宜蘭縣冬山鄉,663,2022
3802323,2022,11,20221113,234548,A1,宜蘭縣政府警察局,宜蘭縣員山鄉員山路2段216號前0.0公尺,121.708366,24.722569,0,...,...,...,...,...,宜蘭縣員山鄉,2022宜蘭縣員山鄉,110,宜蘭縣員山鄉,286,2022
3802324,2022,11,20221113,234548,A1,宜蘭縣政府警察局,宜蘭縣員山鄉員山路2段216號前0.0公尺,121.708366,24.722569,0,...,...,...,...,...,宜蘭縣員山鄉,2022宜蘭縣員山鄉,110,宜蘭縣員山鄉,286,2022
3802325,2022,11,20221114,23500,A1,新北市政府警察局,新北市淡水區新民街324號前0.0公尺,121.435276,25.184107,1,...,...,...,...,NaN,新北市淡水區,2022新北市淡水區,110,新北市淡水區,2621,2022
3802326,2022,11,20221114,23500,A1,新北市政府警察局,新北市淡水區新民街324號前0.0公尺,121.435276,25.184107,1,...,...,...,...,NaN,新北市淡水區,2022新北市淡水區,110,新北市淡水區,2621,2022
3802327,2022,11,20221114,23500,A1,新北市政府警察局,新北市淡水區新民街324號前0.0公尺,121.435276,25.184107,1,...,...,...,...,NaN,新北市淡水區,2022新北市淡水區,110,新北市淡水區,2621,2022
3802328,2022,11,20221114,23500,A1,新北市政府警察局,新北市淡水區新民街324號前0.0公尺,121.435276,25.184107,1,...,...,...,...,NaN,新北市淡水區,2022新北市淡水區,110,新北市淡水區,2621,2022
3802329,2022,11,20221114,141400,A1,新北市政府警察局,新北市林口區文化一路1段 / 新北市林口區麗園一街,121.371712,25.070273,1,...,...,...,...,NaN,新北市林口區,2022新北市林口區,110,新北市林口區,2301,2022
3802330,2022,11,20221114,141400,A1,新北市政府警察局,新北市林口區文化一路1段 / 新北市林口區麗園一街,121.371712,25.070273,1,...,...,...,...,NaN,新北市林口區,2022新北市林口區,110,新北市林口區,2301,2022


In [27]:
index = 0
for column in data.columns:
    print(index, ":  ", column, end='')
    print('\t\t\t', 'Nan值: ', data[column].isnull().sum(),'筆')
    index += 1

0 :   發生年度			 Nan值:  0 筆
1 :   發生月份			 Nan值:  0 筆
2 :   發生日期			 Nan值:  0 筆
3 :   發生時間			 Nan值:  0 筆
4 :   事故類別名稱			 Nan值:  0 筆
5 :   處理單位名稱警局層			 Nan值:  0 筆
6 :   發生地點			 Nan值:  0 筆
7 :   經度			 Nan值:  0 筆
8 :   緯度			 Nan值:  0 筆
9 :   weather_晴			 Nan值:  0 筆
10 :   weather_特殊			 Nan值:  0 筆
11 :   weather_陰			 Nan值:  0 筆
12 :   weather_雨			 Nan值:  0 筆
13 :   light_夜間有照明			 Nan值:  0 筆
14 :   light_夜間無照明			 Nan值:  0 筆
15 :   light_日間自然光線			 Nan值:  0 筆
16 :   light_晨或暮光			 Nan值:  0 筆
17 :   roadtype_交岔路			 Nan值:  0 筆
18 :   roadtype_單路部分			 Nan值:  0 筆
19 :   drywet_乾燥			 Nan值:  0 筆
20 :   drywet_濕潤			 Nan值:  0 筆
21 :   pit_有坑洞			 Nan值:  0 筆
22 :   pit_無缺陷			 Nan值:  0 筆
23 :   obst_有障礙物			 Nan值:  0 筆
24 :   obst_無障礙物			 Nan值:  0 筆
25 :   sign_無號誌			 Nan值:  0 筆
26 :   sign_行車管制號誌			 Nan值:  0 筆
27 :   sign_閃光號誌			 Nan值:  0 筆
28 :   line_中央分向島			 Nan值:  0 筆
29 :   line_無			 Nan值:  0 筆
30 :   line_禁止超車線			 Nan值:  0 筆
31 :   line_行車分向線			 Nan值:  0 筆
32 :   速限_recode			 Nan值:  0 筆
33 :   調查站1_座標_經

# 加入day of week、day of year

In [7]:
pos_df = pd.read_csv("pos_data(no traffic attribute).csv", encoding='utf-8-sig')
pos_df = pos_df.drop_duplicates()

In [8]:
pos_df['發生日期'] = pos_df.發生日期.astype(str).apply(lambda x: x[-2:]).astype(int)

In [9]:
date = pos_df.apply((lambda x: x.發生年度.astype(int).astype(str)+'-'+ x.發生月份.astype(int).astype(str)+'-'+x.發生日期.astype(int).astype(str)), axis = 1)

In [10]:
# 週一為0，週日為6
dow = pd.to_datetime(date).dt.dayofweek
pos_df['dow'] = dow

In [11]:
doy = pd.to_datetime(date).dt.dayofyear
pos_df['doy'] = doy

# 加入107-111假期單熱編碼m

In [12]:
def taiwan_festival(x: int) -> pd.Series:
    
    sr = pd.Series([0,0,0,0,0,0,0],index=['元旦','春節','二二八','兒童清明','端午','中秋','雙十'])
    
    def _2018( x, sr):
        if x == 20180101:
            sr['元旦'] = 1
        elif 20180215 <= x <= 20180220:
            sr['春節'] = 1
        elif x == 20180228:
            sr['二二八'] = 1
        elif 20180404 <= x <= 20180408:
            sr['兒童清明'] = 1
        elif 20180616 <= x <= 20180618:
            sr['端午'] = 1
        elif 20180922 <= x <= 20180924:
            sr['中秋'] = 1
        elif x == 20181010:
            sr['雙十'] = 1
        elif 20181229 <= x <= 20181231:
            sr['元旦'] = 1
        return sr

    def _2019( x, sr):
        if x == 20190101:
            sr['元旦'] = 1
        elif 20190202 <= x <= 20190210:
            sr['春節'] = 1
        elif 20190228 <= x <= 20190303:
            sr['二二八'] = 1
        elif 20190404 <= x <= 20190407:
            sr['兒童清明'] = 1
        elif 20190607 <= x <= 20190609:
            sr['端午'] = 1
        elif 20190913 <= x <= 20190915:
            sr['中秋'] = 1
        elif 20191010 <= x <= 20191013:
            sr['雙十'] = 1
        return sr


    def _2020( x, sr):
        if x == 20200101:
            sr['元旦'] = 1
        elif 20200123 <= x <= 20200129:
            sr['春節'] = 1
        elif 20200228 <= x <= 20200301:
            sr['二二八'] = 1
        elif 20200402 <= x <= 20200405:
            sr['兒童清明'] = 1
        elif 20200625 <= x <= 20200628:
            sr['端午'] = 1
        elif 20201001 <= x <= 20201004:
            sr['中秋'] = 1
        elif 20201009 <= x <= 20201011:
            sr['雙十'] = 1
        return sr

    def _2021( x, sr):
        if 20210101 <= x <= 20210103:
            sr['元旦'] = 1
        elif 20210210  <= x <= 20210216:
            sr['春節'] = 1
        elif 20210227 <= x <= 20210301:
            sr['二二八'] = 1
        elif 20210402 <= x <= 20210405:
            sr['兒童清明'] = 1
        elif 20210612 <= x <= 20210614:
            sr['端午'] = 1
        elif 20210918 <= x <= 20210921:
            sr['中秋'] = 1
        elif 20211009 <= x <= 20211011:
            sr['雙十'] = 1
        elif x == 20211231:
            sr['元旦'] = 1
        return sr

    def _2022( x, sr):
        if 20220101 <= x <= 20220102:
            sr['元旦'] = 1
        elif 20220129 <= x <= 20220206:
            sr['春節'] = 1
        elif 20220226 <= x <= 20220228:
            sr['二二八'] = 1
        elif 20220402 <= x <= 20220405:
            sr['兒童清明'] = 1
        elif 20220603 <= x <= 20220605:
            sr['端午'] = 1
        elif 20220909 <= x <= 20220911:
            sr['中秋'] = 1
        elif 20221008 <= x <= 20221010:
            sr['雙十'] = 1
        elif x == 20221231:
            sr['元旦'] = 1
        return sr

    year = x // 10000
    
    if year == 2018:
        sr = _2018(x, sr)
    elif year == 2019:
        sr = _2019(x, sr)
    elif year == 2020:
        sr = _2020(x, sr)
    elif year == 2021:
        sr = _2021(x, sr)
    elif year == 2022:
        sr = _2022(x, sr)
    
    return sr


In [13]:
date_int = pos_df.發生年度.astype(int)*10000 + pos_df.發生月份.astype(int)*100 + pos_df.發生日期.astype(int)

In [14]:
from tqdm import tqdm
tqdm.pandas()
festival_df = date_int.progress_apply(taiwan_festival)

100%|██████████████████████████████████████████████████████████████████████| 1666630/1666630 [04:14<00:00, 6550.53it/s]


In [15]:
festival_df.sum()

元旦      10731
春節      27506
二二八     11042
兒童清明    16580
端午      11816
中秋      13930
雙十      11321
dtype: int64

In [16]:
pos_df = pd.concat([festival_df, pos_df], axis=1)

In [17]:
order_list = [ i for i in range(pos_df.shape[1])]
order_list = [-2, -1] + order_list[:-2]

In [18]:
pos_df = pos_df.iloc[:,order_list]

In [19]:
pos_df

,dow,doy,元旦,春節,二二八,兒童清明,端午,中秋,雙十,發生年度,...,緯度,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x,population_density,流量加權,車禍發生
0,0,1,1,0,0,0,0,0,0,2018,...,22.999773,14.4,1012.0,99.0,0.0,137.0,0.5,6009,26664.974265,1
3,0,1,1,0,0,0,0,0,0,2018,...,23.026156,17.8,1017.8,84.0,0.0,22.0,2.0,1793,26813.457545,1
5,0,1,1,0,0,0,0,0,0,2018,...,23.061173,16.5,1017.2,90.0,0.0,7.0,0.8,765,22123.122152,1
9,0,1,1,0,0,0,0,0,0,2018,...,22.981113,17.1,1017.0,79.0,0.0,10.0,2.2,13927,23079.971821,1
11,0,1,1,0,0,0,0,0,0,2018,...,22.950055,16.7,1013.7,89.0,0.0,44.0,1.2,1226,16383.479745,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3783382,5,316,0,0,0,0,0,0,0,2022,...,24.655476,18.8,905.1,100.0,0.0,146.0,5.0,663,9022.933719,1
3783384,5,316,0,0,0,0,0,0,0,2022,...,23.372847,25.7,1012.9,76.0,0.0,21.0,2.2,211,9412.394478,1
3783388,6,317,0,0,0,0,0,0,0,2022,...,24.722569,23.5,1016.8,99.0,11.0,239.0,1.2,286,2511.299322,1
3783390,0,318,0,0,0,0,0,0,0,2022,...,25.184107,21.5,1001.6,90.0,0.0,86.0,2.9,2621,7925.824537,1


In [20]:
pos_df.to_csv('pos_data(no traffic attribute + festival).csv',encoding='utf-8-sig',mode='w',index=None)

In [21]:
negitive_df = pd.read_csv('neg_data(no traffic attribute + festival).csv', encoding='utf-8-sig', low_memory=False)

In [22]:
negitive_df

,dow,doy,元旦,春節,二二八,兒童清明,端午,中秋,雙十,發生年度,...,緯度,氣溫(℃)_x,測站氣壓(hPa)_x,相對溼度(%)_x,降水量(mm)_x,風向(360degree)_x,風速(m/s)_x,population_density,流量加權,車禍發生
0,4,353,0,0,0,0,0,0,0,2020,...,23.703466,20.8,1014.0,79.0,0.0,357.0,4.1,1159,13124.064104,0
1,2,125,0,0,0,0,0,0,0,2021,...,25.077870,21.0,1014.2,80.0,0.0,146.0,0.9,4465,9938.311810,0
2,4,295,0,0,0,0,0,0,0,2021,...,24.949437,17.0,990.4,100.0,2.0,31.0,1.7,62,5910.896902,0
3,1,355,0,0,0,0,0,0,0,2021,...,24.633933,17.9,997.9,97.0,0.5,0.0,0.0,19205,19306.252201,0
4,6,97,0,0,0,1,0,0,0,2019,...,22.680644,27.9,1011.1,67.0,0.0,244.0,3.9,3068,15124.156391,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3704727,0,287,0,0,0,0,0,0,0,2019,...,22.551515,26.5,1006.5,84.0,0.0,205.0,0.4,339,8336.942344,0
3704728,2,173,0,0,0,0,0,0,0,2022,...,23.925731,16.3,791.2,99.0,2.0,166.0,0.9,1372,16114.008396,0
3704729,2,267,0,0,0,0,0,0,0,2020,...,23.759012,30.0,1004.4,66.0,0.0,282.0,0.9,473,9085.433554,0
3704730,5,246,0,0,0,0,0,0,0,2022,...,24.032694,11.6,697.4,100.0,2.5,197.0,2.4,932,4610.471958,0


In [23]:
pos_df.to_csv('../2.機器學習/無交通相關特徵+節日+正負三比一資料集/pos_data(no traffic attribute + festival).csv',encoding='utf-8-sig',mode='w',index=None)

In [ ]:
negitive_df